In [166]:
#Importing the necessary packages
import gym 
import tensorflow as tf 
import numpy as np 
import random 
import matplotlib.pyplot as plt

In [167]:
#quick exploration of our environment
env = gym.make('Pong-v0')
print(env.action_space)
print(env.observation_space)


Discrete(6)
Box(210, 160, 3)


In [168]:
class ExperienceBuffer: 
    
    def __init__(self): 
        self.buffer = []
        self.buffer_capacity = 1000000
        self.current_length = 0
    def write(self, obs): 
        if self.current_length == self.buffer_capacity: 
            self.buffer.pop()
            self.current_length -= 1
        self.buffer.append(obs)
        self.current_length +=1
    def sample_buffer(self, batch_to_sample): 
        return random.sample(self.buffer, min(batch_to_sample, self.current_length))
    def get_capacity(self): 
        return self.current_length

In [200]:
class Network:
    
    def __init__(self, session , n_out):
        self.session = session
        self.n_out = n_out
        # data placeholders
        self.x = tf.placeholder(tf.float32, [None, 84,84,1], name='x')
        self.y = tf.placeholder(tf.float32, [None, n_out], name='y')
        self.conv_1 = tf.layers.conv2d(self.x, filters = 32, kernel_size = [8,8], strides = 4, activation = tf.nn.relu)
        self.conv_2 = tf.layers.conv2d(self.conv_1, filters = 64, kernel_size = [4,4], strides = 2, activation = tf.nn.relu)
        #self.max_pool_1 = tf.layers.MaxPooling2D(pool_size = 2, strides = 2)
        self.flat_layer = tf.layers.flatten(self.conv_2)
        self.bottleneck_layer = tf.layers.dense(self.flat_layer, 512)
        self.q = tf.layers.dense(self.bottleneck_layer, n_out)
        #COULD BE BUGGY!
        self.loss = tf.reduce_sum(tf.square(self.y - self.q))
        self.train_step = tf.train.AdamOptimizer(1e-4).minimize(self.loss)
    def compute(self, input_frames):
        # evaluate the network and return the action values [q(s,a=0),q(s,a=1)]
        return self.session.run(self.q, feed_dict={self.x:input_frames})
    def train(self, x_batch, y_batch): 
            _ = self.session.run(self.train_step, feed_dict={self.x: x_batch, self.y: y_batch})

In [170]:
class ReshapeImage: 
    #Source for data processing: this is also the same as the openAI baseline https://github.com/fg91/Deep-Q-Learning/blob/master/DQN.ipynb
    def __init__(self, target_height = 84, target_width = 84):
        self.target_height = target_height
        self.target_width = target_width
        self.input_rgb_image = tf.placeholder(tf.uint8, [210,160,3])
        self.grayscale_img = tf.image.rgb_to_grayscale(self.input_rgb_image)
        self.bounded_img = tf.image.crop_to_bounding_box(self.grayscale_img, 34, 0, 160, 160)
        self.final_processed = tf.image.resize_images(self.bounded_img, 
                                                [self.target_height, self.target_width], 
                                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    def return_processed_img(self, session, input_img):
        return session.run(self.final_processed, feed_dict = {self.input_rgb_image: input_img})

In [191]:
class Agent: 
    
    def __init__(self, session, policy = 'DQN'): 
        #Want a pretty high gamma at first
        self.gamma = 0.99
        #Want pure exploration at first 
        self.epsilon = 1.0
        #want massive adjustments to Q function at first
        self.alpha = 1.0
        #There are six possible outputs
        self.n_out = 6
        #Tune this
        self.batch_size = 32
        self.total_reward = 0
        self.policy = policy
        self.session = session
        self.experience_buffer = ExperienceBuffer()
        self.q = Network(session, self.n_out)
        self.image_reshaper = ReshapeImage()
    def get_action(self, observation): 
        if self.policy == 'DQN':
            if np.random.randn() > self.epsilon: 
                #reshaped_obs = self.image_reshaper.return_processed_img(self.session, observation)
                #need to fix this to feed in 4 stacked frames
               # pos = np.argmax(self.q.compute(reshaped_obs), axis = 1)
                #return pos[0];
                print('use dqn')
                processed_obv = self.image_reshaper.return_processed_img(self.session, observation)
                padded_processed = np.expand_dims(processed_obv, axis = 0)
                action_values = self.q.compute(padded_processed)
                max_action = np.argmax(action_values, axis = 1)
                return max_action[0]
            else: 
                print('chose randomly')
                return random.randint(0,5)
    def adjust_epsilon(self, episode_number):
        self.epsilon = 0.01 + (1.0 - 0.01)*np.exp(-.001*episode_number)
    def update_q(): 
        self.batch_to_train_upon = self.experience_buffer.sample_buffer(self.batch_size);
    def gather_buffer_exp(self, last_obs, action, reward, obs): 
        processed_obv = self.image_reshaper.return_processed_img(self.session, obs)
        processed_last_obv = self.image_reshaper.return_processed_img(self.session, last_obs)
        self.experience_buffer.write((processed_last_obv, action, reward, processed_obv))
    def gather_reward(self, reward): 
        self.total_reward += reward
    def get_total_reward(self): 
        return self.total_reward
    def set_reward(self, new_reward): 
        self.total_reward = new_reward
    def return_buffer_size(self): 
        return self.experience_buffer.get_capacity()
    def update_q(self): 
        #PRETTY SURE THIS IS BUSTED BUT FIX LATER!
        self.sars_buffer = self.experience_buffer.sample_buffer(self.batch_size)
        the_x_batch = np.zeros(shape = (np.shape(self.sars_buffer)[0], 84,84,1))
        the_y_batch = np.zeros(shape = (np.shape(self.sars_buffer)[0], self.n_out))
        last_stateq = np.zeros(shape = (np.shape(self.sars_buffer)[0], self.n_out))
        next_q = np.zeros(shape = (np.shape(self.sars_buffer)[0], self.n_out))
        for i in range(np.shape(self.sars_buffer)[0]):
            padded_processed = np.expand_dims(self.sars_buffer[i][0], axis = 0)
            p2 = np.expand_dims(self.sars_buffer[i][3], axis = 0)
            last_stateq[i, :] = self.q.compute(padded_processed)
            next_q[i,:] = self.q.compute(p2)
        for i in range(np.shape(self.sars_buffer)[0]): 
            the_x_batch[i, :] = self.sars_buffer[i][0]
            for j in range(6): 
                if j == self.sars_buffer[i][1]:
                     #action we need to update
                    the_y_batch[i, j] = self.sars_buffer[i][2] + self.gamma * np.max(next_q[i])
                else: 
                     #use the previous q value
                    the_y_batch[i,j] = last_stateq[i,j]
                    
        self.q.train(the_x_batch, the_y_batch)
        print(self.q.)

In [192]:
the_x_batch = np.zeros(shape = (32, 84,84,1))
for i in range(32): 
    the_x_batch[i, :] = np.random.rand(84,84,1)
print(np.shape(the_x_batch))

(32, 84, 84, 1)


In [194]:
#quick test of the agents functions
agent = Agent('random')
image = np.random.rand(210,160,3)
act = agent.get_action(image)
new_rew = agent.gather_reward(15)
new_tot = agent.get_total_reward()
print(new_tot)
print(act)

chose randomly
15
1


In [198]:
#Code to test the actual env:
with tf.Graph().as_default():
    with tf.Session() as session: 
        agent = Agent(session)
        session.run(tf.global_variables_initializer())
        episode_rewards = []
        for i_episode in range(20):
            observation = env.reset()
            agent.set_reward(0)
            for t in range(4000):
                flag = True
                env.render()
                action = agent.get_action(observation)
                new_observation, reward, done, info = env.step(action)
                if done: 
                    new_observation = None
                    flag = False
                print("the reward at stage {} was {} and the action was {}".format(t, reward, action))
                agent.gather_reward(reward)
                if flag: 
                    agent.gather_buffer_exp(observation, action, reward, new_observation)
                agent.update_q()
                observation = new_observation
                if done:
                    the_ep_reward = agent.get_total_reward()
                    print("Episode {} finished after {} timesteps with total reward {}".format(i_episode, t+1, agent.get_total_reward()))
                    episode_rewards.append(the_ep_reward)
                    print("current buffer size is {}".format(agent.return_buffer_size()))
                    break
            agent.adjust_epsilon(i_episode)
plt.plot(episode_rewards)
plt.show()

chose randomly
the reward at stage 0 was 0.0 and the action was 1
chose randomly
the reward at stage 1 was 0.0 and the action was 5
chose randomly
the reward at stage 2 was 0.0 and the action was 2
chose randomly
the reward at stage 3 was 0.0 and the action was 2
chose randomly
the reward at stage 4 was 0.0 and the action was 1
chose randomly
the reward at stage 5 was 0.0 and the action was 0
chose randomly
the reward at stage 6 was 0.0 and the action was 2
chose randomly
the reward at stage 7 was 0.0 and the action was 3
chose randomly
the reward at stage 8 was 0.0 and the action was 2
chose randomly
the reward at stage 9 was 0.0 and the action was 1
chose randomly
the reward at stage 10 was 0.0 and the action was 0
chose randomly
the reward at stage 11 was 0.0 and the action was 5
chose randomly
the reward at stage 12 was 0.0 and the action was 4
use dqn
the reward at stage 13 was 0.0 and the action was 0
use dqn
the reward at stage 14 was 0.0 and the action was 0
chose randomly
the 

chose randomly
the reward at stage 124 was 0.0 and the action was 4
chose randomly
the reward at stage 125 was 0.0 and the action was 5
chose randomly
the reward at stage 126 was 0.0 and the action was 5
chose randomly
the reward at stage 127 was 0.0 and the action was 0
chose randomly
the reward at stage 128 was 0.0 and the action was 4
chose randomly
the reward at stage 129 was 0.0 and the action was 3
use dqn
the reward at stage 130 was 0.0 and the action was 1
chose randomly
the reward at stage 131 was -1.0 and the action was 0
chose randomly
the reward at stage 132 was 0.0 and the action was 1
chose randomly
the reward at stage 133 was 0.0 and the action was 1
use dqn
the reward at stage 134 was 0.0 and the action was 5
chose randomly
the reward at stage 135 was 0.0 and the action was 4
chose randomly
the reward at stage 136 was 0.0 and the action was 4
chose randomly
the reward at stage 137 was 0.0 and the action was 0
chose randomly
the reward at stage 138 was 0.0 and the action

chose randomly
the reward at stage 247 was 0.0 and the action was 3
chose randomly
the reward at stage 248 was 0.0 and the action was 0
chose randomly
the reward at stage 249 was 0.0 and the action was 1
chose randomly
the reward at stage 250 was 0.0 and the action was 3
chose randomly
the reward at stage 251 was 0.0 and the action was 4
chose randomly
the reward at stage 252 was 0.0 and the action was 1
chose randomly
the reward at stage 253 was 0.0 and the action was 4
chose randomly
the reward at stage 254 was 0.0 and the action was 0
chose randomly
the reward at stage 255 was 0.0 and the action was 4
use dqn
the reward at stage 256 was 0.0 and the action was 3
chose randomly
the reward at stage 257 was 0.0 and the action was 0
chose randomly
the reward at stage 258 was 0.0 and the action was 2
chose randomly
the reward at stage 259 was 0.0 and the action was 4
chose randomly
the reward at stage 260 was 0.0 and the action was 1
use dqn
the reward at stage 261 was 0.0 and the action 

chose randomly
the reward at stage 369 was 0.0 and the action was 2
chose randomly
the reward at stage 370 was 0.0 and the action was 1
chose randomly
the reward at stage 371 was 0.0 and the action was 1
chose randomly
the reward at stage 372 was 0.0 and the action was 3
use dqn
the reward at stage 373 was 0.0 and the action was 2
chose randomly
the reward at stage 374 was 0.0 and the action was 4
chose randomly
the reward at stage 375 was 0.0 and the action was 3
chose randomly
the reward at stage 376 was 0.0 and the action was 2
use dqn
the reward at stage 377 was 0.0 and the action was 2
chose randomly
the reward at stage 378 was 0.0 and the action was 5
use dqn
the reward at stage 379 was 0.0 and the action was 3
chose randomly
the reward at stage 380 was 0.0 and the action was 2
use dqn
the reward at stage 381 was 0.0 and the action was 0
use dqn
the reward at stage 382 was 0.0 and the action was 0
chose randomly
the reward at stage 383 was 0.0 and the action was 1
chose randomly


chose randomly
the reward at stage 492 was 0.0 and the action was 2
use dqn
the reward at stage 493 was 0.0 and the action was 2
chose randomly
the reward at stage 494 was 0.0 and the action was 4
chose randomly
the reward at stage 495 was 0.0 and the action was 4
chose randomly
the reward at stage 496 was 0.0 and the action was 5
chose randomly
the reward at stage 497 was 0.0 and the action was 5
chose randomly
the reward at stage 498 was 0.0 and the action was 5
chose randomly
the reward at stage 499 was 0.0 and the action was 4
use dqn
the reward at stage 500 was 0.0 and the action was 3
chose randomly
the reward at stage 501 was 0.0 and the action was 4
chose randomly
the reward at stage 502 was 0.0 and the action was 1
use dqn
the reward at stage 503 was 0.0 and the action was 3
use dqn
the reward at stage 504 was -1.0 and the action was 3
chose randomly
the reward at stage 505 was 0.0 and the action was 2
chose randomly
the reward at stage 506 was 0.0 and the action was 3
chose r

...

use dqn
the reward at stage 326 was 0.0 and the action was 3
chose randomly
the reward at stage 327 was 0.0 and the action was 1
chose randomly
the reward at stage 328 was 0.0 and the action was 4
chose randomly
the reward at stage 329 was 0.0 and the action was 0
chose randomly
the reward at stage 330 was 0.0 and the action was 3
chose randomly
the reward at stage 331 was 0.0 and the action was 3
use dqn
the reward at stage 332 was 0.0 and the action was 5
chose randomly
the reward at stage 333 was 0.0 and the action was 5
chose randomly
the reward at stage 334 was 0.0 and the action was 4
chose randomly
the reward at stage 335 was 0.0 and the action was 0
chose randomly
the reward at stage 336 was 0.0 and the action was 1
use dqn
the reward at stage 337 was 0.0 and the action was 0
chose randomly
the reward at stage 338 was 0.0 and the action was 3
chose randomly
the reward at stage 339 was 0.0 and the action was 1
chose randomly
the reward at stage 340 was 0.0 and the action was 4
c

chose randomly
the reward at stage 449 was 0.0 and the action was 3
chose randomly
the reward at stage 450 was 0.0 and the action was 1
chose randomly
the reward at stage 451 was 0.0 and the action was 3
chose randomly
the reward at stage 452 was 0.0 and the action was 2
chose randomly
the reward at stage 453 was 0.0 and the action was 0
use dqn
the reward at stage 454 was 0.0 and the action was 4
chose randomly
the reward at stage 455 was 0.0 and the action was 4
chose randomly
the reward at stage 456 was 0.0 and the action was 2
chose randomly
the reward at stage 457 was 0.0 and the action was 4
chose randomly
the reward at stage 458 was 0.0 and the action was 1
chose randomly
the reward at stage 459 was 0.0 and the action was 4
chose randomly
the reward at stage 460 was 0.0 and the action was 0
chose randomly
the reward at stage 461 was 0.0 and the action was 5
chose randomly
the reward at stage 462 was 0.0 and the action was 1
chose randomly
the reward at stage 463 was 0.0 and the 

use dqn
the reward at stage 571 was 0.0 and the action was 3
chose randomly
the reward at stage 572 was 0.0 and the action was 4
use dqn
the reward at stage 573 was 0.0 and the action was 1
use dqn
the reward at stage 574 was 0.0 and the action was 1
chose randomly
the reward at stage 575 was 0.0 and the action was 5
chose randomly
the reward at stage 576 was 0.0 and the action was 5
chose randomly
the reward at stage 577 was 0.0 and the action was 5
chose randomly
the reward at stage 578 was 0.0 and the action was 0
use dqn
the reward at stage 579 was 0.0 and the action was 5
chose randomly
the reward at stage 580 was 0.0 and the action was 2
chose randomly
the reward at stage 581 was 0.0 and the action was 3
chose randomly
the reward at stage 582 was 0.0 and the action was 1
chose randomly
the reward at stage 583 was 0.0 and the action was 1
chose randomly
the reward at stage 584 was 0.0 and the action was 3
use dqn
the reward at stage 585 was 0.0 and the action was 4
chose randomly


chose randomly
the reward at stage 694 was 0.0 and the action was 4
chose randomly
the reward at stage 695 was 0.0 and the action was 1
chose randomly
the reward at stage 696 was 0.0 and the action was 0
chose randomly
the reward at stage 697 was 0.0 and the action was 5
use dqn
the reward at stage 698 was 0.0 and the action was 3
chose randomly
the reward at stage 699 was 0.0 and the action was 2
use dqn
the reward at stage 700 was 0.0 and the action was 2
chose randomly
the reward at stage 701 was 0.0 and the action was 0
use dqn
the reward at stage 702 was 0.0 and the action was 5
chose randomly
the reward at stage 703 was 0.0 and the action was 5
chose randomly
the reward at stage 704 was 0.0 and the action was 3
chose randomly
the reward at stage 705 was 0.0 and the action was 3
use dqn
the reward at stage 706 was 0.0 and the action was 4
chose randomly
the reward at stage 707 was 0.0 and the action was 3
chose randomly
the reward at stage 708 was 0.0 and the action was 4
chose ra

KeyboardInterrupt: 

In [103]:
#To test the image reshaper
with tf.Graph().as_default():
    with tf.Session() as sess: 
        reshaper = ReshapeImage()
        x = np.random.rand(210,160,3)
        fin_img = reshaper.return_processed_img(sess, x)
        print(fin_img.shape)

(84, 84, 1)


In [73]:
#to test network 
with tf.Graph().as_default(): 
    with tf.Session() as session:
        x = np.random.rand(84,84,1)
        x = np.expand_dims(x, axis = 0)
        the_net = Network(session, 6)
        session.run(tf.global_variables_initializer())
        q_func = the_net.compute(x)
        print(q_func.shape)
        
       # y =np.array([ x[:,:,:,2]+x[:,:, :, 3]**2, x[:,:,:, 1]+x[:,:, :, 0]**3, x[:,:,:,2]+x[:,:, :, 3]**2,  x[:,:,:,2]+x[:,:, :, 3]**2,  x[:,:,:,2]+x[:,:, :, 3]**2,  x[:,:,:,2]+x[:,:, :, 3]**2 ])
            
       # print('MSE at iteration 0 is {}'.format(((the_net.compute(x) - y)**2).mean()))
        
        # now train...
       # for i in range(10000):
       #     f.train(x,y)
        
       # print('MSE at iteration 10000 is {}'.format(((f.compute(x) - y)**2).mean()))

(1, 6)


In [100]:
#Test to make sure the qn pick was working
with tf.Graph().as_default(): 
    with tf.Session() as sesh: 
        agent = Agent(sesh)
        sesh.run(tf.global_variables_initializer())
        obs = np.random.rand(210,160,3)
        xD = agent.get_action(obs)
        print(xD)

chose randomly
2


In [159]:
lol = ExperienceBuffer()
for i in range(300): 
    x = np.random.rand(84,84,4)
    y = np.random.randn()
    z = np.random.randn()
    w = np.random.rand(84,84,4)
    lol.write((x,y,z,w))
zed= lol.sample_buffer(32)
print(np.shape(zed))
print(np.shape(zed[0][0]))


(32, 4)
(84, 84, 4)


In [164]:
kek = np.zeros(shape = (32,6))
for i in range(32): 
    y = np.random.rand(1,6)
    kek[i, :] = y
print(kek)
print(kek[2][1], kek[2,1])
print(y)

[[0.99967857 0.77755045 0.39928826 0.82155574 0.46498997 0.50541124]
 [0.98858327 0.56003658 0.34465202 0.95202025 0.64198322 0.35646291]
 [0.848343   0.7387312  0.49615047 0.86692792 0.44064274 0.67457604]
 [0.16686601 0.93451754 0.96892995 0.42989393 0.45780551 0.28353687]
 [0.07007438 0.31176035 0.52442996 0.2793535  0.50522248 0.34053703]
 [0.7119456  0.84195932 0.61100135 0.56940762 0.42746073 0.17801564]
 [0.61395426 0.23361081 0.89400419 0.32500649 0.51124376 0.24193862]
 [0.54912534 0.15479273 0.16868539 0.27787072 0.43491388 0.65655194]
 [0.68352529 0.75305926 0.69077675 0.08718559 0.89251229 0.8954248 ]
 [0.67721696 0.48735854 0.6927927  0.49487113 0.13328808 0.17924495]
 [0.91752592 0.58600743 0.40714383 0.16409967 0.91951931 0.98202629]
 [0.07899979 0.95399712 0.2748201  0.87785284 0.52908302 0.53984104]
 [0.34857035 0.59143641 0.07711803 0.95114209 0.82819717 0.50583815]
 [0.74169844 0.94640898 0.44916519 0.39936731 0.58609137 0.56013468]
 [0.90679047 0.43242194 0.57378314

In [199]:
#Figure out 4-frame stack: current idea let obs = [x,y,z,w] update newobs [y,z,w,new] and let the action reward pair
np.random.rand(10,10,4)

array([[[0.45667416, 0.19330643, 0.29983587, 0.88837206],
        [0.57872627, 0.87666141, 0.92983322, 0.79583591],
        [0.8290084 , 0.51227233, 0.02832566, 0.31192551],
        [0.03043676, 0.98655067, 0.83488915, 0.04634185],
        [0.01208752, 0.52871864, 0.88182093, 0.58292668],
        [0.15753831, 0.16977441, 0.22566735, 0.18448366],
        [0.82958214, 0.57382163, 0.6926248 , 0.2896076 ],
        [0.4720349 , 0.40958407, 0.86209286, 0.74987907],
        [0.15890764, 0.96147645, 0.93574238, 0.93078311],
        [0.9145597 , 0.51774076, 0.63100785, 0.41831544]],

       [[0.06498345, 0.31678346, 0.01535584, 0.44949588],
        [0.63424654, 0.71975778, 0.77526492, 0.3689303 ],
        [0.61890323, 0.78126262, 0.40601726, 0.15834915],
        [0.17536133, 0.44770235, 0.53971149, 0.07308775],
        [0.79425979, 0.06090749, 0.18316339, 0.70166503],
        [0.72985799, 0.08608247, 0.54933593, 0.27464197],
        [0.75904455, 0.88120842, 0.40272505, 0.5727806 ],
        [0.3